In [1]:
! pip install kashgari
! pip install pandas
# ! pip install tensorflow==1.15.0
%tensorflow_version 1.x


     |████████████████████████████████| 92kB 2.4MB/s 
     |████████████████████████████████| 17.3MB 1.3MB/s 
  Created wheel for keras-gpt-2: filename=keras_gpt_2-0.14.0-cp36-none-any.whl size=10525 sha256=13fd9210e2c9da8dd32adc7aac623a4dc985c42dec8ec5e4143e8d361f6d336e
  Stored in directory: /root/.cache/pip/wheels/ec/d8/06/ba8216a77a55b8ba4a5c3932c7df93e87eeaea83ced27822aa
  Created wheel for bert4keras: filename=bert4keras-0.6.5-cp36-none-any.whl size=34212 sha256=3caf6749e026c2c476ecab43510e28b9e7e7df8649356c4415a5ff393b6c4ccc
  Stored in directory: /root/.cache/pip/wheels/97/82/72/56a894ccb2337a25b679cbac552bf2f15f5e8e798a37313de6
  Created wheel for keras-bert: filename=keras_bert-0.84.0-cp36-none-any.whl size=36139 sha256=ab1f5505b51dce29ee2edb502b92b5ad7d312e0ff568aeae1073ffcbc52b0da1
  Stored in directory: /root/.cache/pip/wheels/1f/59/04/12e95257aebfd27f7edaaf65ab7dd57b5f6cadfb183f1a4ccd
  Created wheel for keras-transformer: filename=keras_transformer-0.37.0-cp36-none-any.w

TensorFlow 1.x selected.


In [4]:
import pandas as pd
import kashgari
import re
import numpy

#  加載路徑下的模型::  此次模型名稱為recall85_rbt3
model_path = "drive/My Drive/"
loaded_model = kashgari.utils.load_model(model_path+'recall85_rbt3')

/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:22: UserWarning: bert4keras.bert has been renamed as bert4keras.models.
  warnings.warn('bert4keras.bert has been renamed as bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:23: UserWarning: please use bert4keras.models.
  warnings.warn('please use bert4keras.models.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:44: UserWarning: bert4keras.tokenizer has been renamed as bert4keras.tokenizers.
  warnings.warn('bert4keras.tokenizer has been renamed as bert4keras.tokenizers.')
/usr/local/lib/python3.6/dist-packages/bert4keras/__init__.py:45: UserWarning: please use bert4keras.tokenizers.
  warnings.warn('please use bert4keras.tokenizers.')


In [0]:
import kashgari
import re

# loaded_model = kashgari.utils.load_model('per_ner.h5')


def cut_text(text, lenth):
    textArr = re.findall('.{' + str(lenth) + '}', text)
    textArr.append(text[(len(textArr) * lenth):])
    return textArr


def extract_labels(text, ners):
    ner_reg_list = []
    if ners:
        new_ners = []
        for ner in ners:
            new_ners += ner;
        for word, tag in zip([char for char in text], new_ners):
            if tag != 'O':
                ner_reg_list.append((word, tag))

    # 输出模型的NER識別结果
    labels = {}
    if ner_reg_list:
        for i, item in enumerate(ner_reg_list):
            if item[1].startswith('B'):
                label = ""
                end = i + 1
                while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                    end += 1

                ner_type = item[1].split('-')[1]
   
                if ner_type not in labels.keys():
                    labels[ner_type] = []
                
                label += ''.join([item[0] for item in ner_reg_list[i:end]])
                labels[ner_type].append(label)    
                
    return labels


while True:
    text_input = input('sentence: ')

    texts = cut_text(text_input, 100)
    ners = loaded_model.predict([[char for char in text] for text in texts])
    print(ners)
    labels = extract_labels(text_input, ners)
    print(labels)


sentence: 這家店的服務真的超差，總體來講不推薦。
[['O', 'O', 'O', 'O', 'B-1', 'I-1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
{'1': ['服務']}
sentence: 這家店的服務真的超差，整體來講不推薦。
[['O', 'O', 'O', 'O', 'B-1', 'I-1', 'O', 'O', 'O', 'O', 'O', 'B-5', 'I-5', 'O', 'O', 'O', 'O', 'O', 'O']]
{'1': ['服務'], '5': ['整體']}
sentence: 雖然這家店布置得很差，店內又有異味，東西又難吃，但服務生還蠻正的，所以超推這家
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-6', 'I-6', 'O', 'O', 'O', 'O', 'O', 'B-1', 'I-1', 'I-1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
{'6': ['東西'], '1': ['服務生']}
sentence: 雖然這家店裝潢的得很差，店內又有異味，東西又難吃，但服務生還蠻正的，所以超推這家
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-6', 'I-6', 'O', 'O', 'O', 'O', 'O', 'B-1', 'I-1', 'I-1', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]
{'6': ['東西'], '1': ['服務生']}
sentence: 雖然這家店的用餐環境很差，店內又有異味，東西又難吃，但服務生還蠻正的，所以超推這家
[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-2', 'I-2', 'O', 

In [2]:



#請載資料夾內的rbt3_model模型並解壓縮，才能用這個預測菜名的模型

import pandas as pd
import kashgari
import re
import numpy



#定義類別(要用裡面的save_all_col方法才能將輸入的dataframe預測出實體後變成新的dataframe輸出~~)
class Get_dishes:

    global loaded_model

    def __init__(self,df):
        self.D = df
    #re.findall() = 找出字串中含有'.{' + str(lenth) + '}'的所有字串
    def cut_text(self,text, lenth):
        textArr = re.findall('.{' + str(lenth) + '}', text)
        textArr.append(text[(len(textArr) * lenth):])
        return textArr


    def extract_labels(self,text, ners):
        ner_reg_list = []
        if ners:
            new_ners = []
            for ner in ners:
                new_ners += ner;
            for word, tag in zip([char for char in text], new_ners):
                if tag != 'O':
                    ner_reg_list.append((word, tag))

        # 輸出模型的NER识别结果
        labels = {}
        if ner_reg_list:
            for i, item in enumerate(ner_reg_list):
                if item[1].startswith('B'):
                    label = ""
                    end = i + 1
                    while end <= len(ner_reg_list) - 1 and ner_reg_list[end][1].startswith('I'):
                        end += 1

                    ner_type = item[1].split('-')[1]
                    if ner_type not in labels.keys():
                        labels[ner_type] = []

                    label += ''.join([item[0] for item in ner_reg_list[i:end]])
                    labels[ner_type].append(label)

        return labels

    #建立一個能讓dataframe中的評論欄位,能預測菜單的函式
    def get_dish_name(self,sentence):

        text_1 = self.cut_text(sentence, 100)
        ners = loaded_model.predict([[char for char in text] for text in text_1])
        labels = self.extract_labels(sentence, ners)
        x = labels.get("0",[])
        if len(x) == 0:
          return np.nan
        else:
          return ",".join(x)

    #建立一個能讓dataframe中的評論欄位,能預測服務的函式
    def get_service_name(self,sentence):

        text_1 = self.cut_text(sentence, 100)
        ners = loaded_model.predict([[char for char in text] for text in text_1])
        labels = self.extract_labels(sentence, ners)
        x = labels.get("1",[])
        if len(x) == 0:
          return np.nan
        else:
          return ",".join(x)


    def get_dic(self,sentence):

        text_1 = self.cut_text(sentence, 100)
        ners = loaded_model.predict([[char for char in text] for text in text_1])
        labels = self.extract_labels(sentence, ners)
        return labels
        # if len(dics) == 0:
        #   return np.nan
        # else:
        #   return ",".join(dics)

    def add_all(self,row):

      dic = self.get_dic(row["text"])
      lst_t = ["dish",	"service",	"environment",	"cp",	"traffic",	"general",	"food"]

      for key,lst_value in dic.items():
      
      
        col_name = lst_t[int(key)]
        if len(lst_value) == 0:
          row[col_name] = np.nan
        else:
          row[col_name] = ",".join(lst_value) 
      return row
    
    def save_all_col(self):
      lst_t = ["dish",	"service",	"environment",	"cp",	"traffic",	"general",	"food"]
      self.D[lst_t] = self.D[lst_t].astype(object)
      for index, row in self.D.iterrows():
        dic = self.get_dic(row["text"])
        for key,lst_value in dic.items():
          col_name = lst_t[int(key)]
          if len(lst_value) == 0:
            col_value = np.nan
          else:
            col_value = ",".join(lst_value)
          self.D.at[index,col_name] = col_value 
      return self.D

    #修改dish欄位，放入預測的菜單
    def Get(self):
        self.D.apply(self.add_all,axis=1)
        print(type(self.D))
        return self.D

In [6]:
#以下為測試一筆檔案的程式碼
import time
import numpy as np


start =time.time() 
df = pd.read_csv("新莊飲食堂熱炒_review.csv")
x = Get_dishes(df)
y = x.save_all_col()
end =time.time() 


running_time = (end-start)


print('time cost : %.5f sec' %running_time)

time cost : 19.73760 sec


In [7]:
y.head()

,id,name,date,star,text,dish,service,environment,cp,traffic,general,food
0,1,新莊飲食堂熱炒,2015-05-31,5,"""東西好吃又便宜""",NaN,NaN,NaN,NaN,NaN,NaN,東西
1,2,新莊飲食堂熱炒,2016-05-31,5,"""超級好吃的。。。大家快來 😊""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,新莊飲食堂熱炒,2016-05-31,5,"""超好吃～鹹蛋苦瓜一定要試試看！""",鹹蛋苦瓜,NaN,NaN,NaN,NaN,NaN,NaN
3,4,新莊飲食堂熱炒,2016-05-31,5,""" 超好吃的😋👍，，""",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,新莊飲食堂熱炒,2016-05-31,5,"""都是家常菜口味 價格又划算！每一道菜色都值得嘗試！""",NaN,NaN,NaN,價格,NaN,NaN,"口味,菜色"
